##### Welcome Everyone!

##### My name is Carlos Jones and today I am showing how to do analysis on Unsupervised data using an ruled based Machine Learning algorithm called Association Rule.

##### In this notebook, wI am looking at a small Dataset consisting of 20 transactions from a supermarket. My goal is to provide a list of items that are purchased together, how often they are purchased together, and how confident I am that they will be purchased together.

##### In order to do this, I will apply two Machine Learning algorithm rules called Apriori Rule and Association Rule. With these rules, I will be able to find the Support (frequency) that each item is bought, the Confidence that I have in an item being bought given that a previous item is bought, and the Lift (strength of my Confidence) for each itemset.

##### I have listed instructions and reasoning for each step that I take below, so please follow along throughout the entire notebook.

##### Enjoy and be sure to visit my GitHub page: github.com/CarlosBusiness

##### Contact Information:
- LinkedIN: www.linkedin.com/in/CarlosJonesData
- Twitter: Los The Scientist
- Email: carlosjones.business@gmail.com


##### Before I begin, we want to make sure we import all of the libraries that I will be working with beforehand. Below are a list that I will be working with: 
- pandas 
- numpy
- matplotlib.pyplot
- seaborn
- mlxtend.frequent_patterns

##### Once I import these, I can begin working on my dataset.

In [1]:
import pandas as pd #dataframe package
import numpy as np #array package
import matplotlib.pyplot as plt #plotting package
import seaborn as sns #visualization package

In [2]:
from mlxtend.frequent_patterns import apriori # frequently occuring items/itemsets
from mlxtend.frequent_patterns import association_rules # association rule

##### Next I will import my data, which is in CSV format

In [38]:
grocery = pd.read_csv('GroceryStoreDataSet.csv')

Let's check just a few columns of my dataset 

In [39]:
grocery.head(3)

,"MILK,BREAD,BISCUIT"
0,"BREAD,MILK,BISCUIT,CORNFLAKES"
1,"BREAD,TEA,BOURNVITA"
2,"JAM,MAGGI,BREAD,MILK"


##### Lets check the shape of my dataset in order to see how many rows and columns I have

In [7]:
grocery.shape

(19, 1)

##### Do I have any missing values in this dataset?

In [15]:
grocery.isnull()

,"MILK,BREAD,BISCUIT"
0,False
1,False
2,False
3,False
4,False
5,False
6,False
7,False
8,False
9,False


##### I have noticed that my column name has a funky title "MILK,BREAD,BISCUIT". Let's simplify this column name to just 'products'.

In [18]:
grocery.rename(columns={'MILK,BREAD,BISCUIT':'products'}, inplace=True)

##### Now let's check to make sure that column name has changed and check to see all of our values

In [19]:
grocery.columns

Index(['products'], dtype='object')

In [12]:
grocery.values

array([['BREAD,MILK,BISCUIT,CORNFLAKES'],
       ['BREAD,TEA,BOURNVITA'],
       ['JAM,MAGGI,BREAD,MILK'],
       ['MAGGI,TEA,BISCUIT'],
       ['BREAD,TEA,BOURNVITA'],
       ['MAGGI,TEA,CORNFLAKES'],
       ['MAGGI,BREAD,TEA,BISCUIT'],
       ['JAM,MAGGI,BREAD,TEA'],
       ['BREAD,MILK'],
       ['COFFEE,COCK,BISCUIT,CORNFLAKES'],
       ['COFFEE,COCK,BISCUIT,CORNFLAKES'],
       ['COFFEE,SUGER,BOURNVITA'],
       ['BREAD,COFFEE,COCK'],
       ['BREAD,SUGER,BISCUIT'],
       ['COFFEE,SUGER,CORNFLAKES'],
       ['BREAD,SUGER,BOURNVITA'],
       ['BREAD,COFFEE,SUGER'],
       ['BREAD,COFFEE,SUGER'],
       ['TEA,MILK,COFFEE,CORNFLAKES']], dtype=object)

##### Now I have come to a very interesting part of the dataset. In order for me to apply the Apriori and Association Rules, I need to have each item represent a column, each index (or row) will represent a customer's transaction, and the values will be numerical. Essentially what I will create are 'Boolean' variables which will denote the following:

- 0 = False/ Customer DID NOT purchase
- 1 = True/ Customer DID purchase
    
##### An easy way to do this is to use the Lambda and Apply function. Let me walk you through the steps I am taking:
- Split the data on the comma punctiation that is currently seperating each item
- Import a package called 'mlxtend.preprocessing', in order to move the items from values to columns
- Change the value type from 'object' to 'integer'

In [20]:
grocery = list(grocery["products"].apply(lambda x:x.split(',')))
grocery

[['BREAD', 'MILK', 'BISCUIT', 'CORNFLAKES'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['JAM', 'MAGGI', 'BREAD', 'MILK'],
 ['MAGGI', 'TEA', 'BISCUIT'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['MAGGI', 'TEA', 'CORNFLAKES'],
 ['MAGGI', 'BREAD', 'TEA', 'BISCUIT'],
 ['JAM', 'MAGGI', 'BREAD', 'TEA'],
 ['BREAD', 'MILK'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'SUGER', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'COCK'],
 ['BREAD', 'SUGER', 'BISCUIT'],
 ['COFFEE', 'SUGER', 'CORNFLAKES'],
 ['BREAD', 'SUGER', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'SUGER'],
 ['BREAD', 'COFFEE', 'SUGER'],
 ['TEA', 'MILK', 'COFFEE', 'CORNFLAKES']]

In [21]:
from mlxtend.preprocessing import TransactionEncoder

In [24]:
te = TransactionEncoder()
te_data = te.fit(grocery).transform(grocery)
grocery2 = pd.DataFrame(te_data,columns=te.columns_)
grocery2

,BISCUIT,BOURNVITA,BREAD,COCK,COFFEE,CORNFLAKES,JAM,MAGGI,MILK,SUGER,TEA
0,True,False,True,False,False,True,False,False,True,False,False
1,False,True,True,False,False,False,False,False,False,False,True
2,False,False,True,False,False,False,True,True,True,False,False
3,True,False,False,False,False,False,False,True,False,False,True
4,False,True,True,False,False,False,False,False,False,False,True
5,False,False,False,False,False,True,False,True,False,False,True
6,True,False,True,False,False,False,False,True,False,False,True
7,False,False,True,False,False,False,True,True,False,False,True
8,False,False,True,False,False,False,False,False,True,False,False
9,True,False,False,True,True,True,False,False,False,False,False


In [25]:
grocery2=grocery2.astype('int')

In [26]:
grocery2

,BISCUIT,BOURNVITA,BREAD,COCK,COFFEE,CORNFLAKES,JAM,MAGGI,MILK,SUGER,TEA
0,1,0,1,0,0,1,0,0,1,0,0
1,0,1,1,0,0,0,0,0,0,0,1
2,0,0,1,0,0,0,1,1,1,0,0
3,1,0,0,0,0,0,0,1,0,0,1
4,0,1,1,0,0,0,0,0,0,0,1
5,0,0,0,0,0,1,0,1,0,0,1
6,1,0,1,0,0,0,0,1,0,0,1
7,0,0,1,0,0,0,1,1,0,0,1
8,0,0,1,0,0,0,0,0,1,0,0
9,1,0,0,1,1,1,0,0,0,0,0


##### My next step is just for curiosity. I want to know a count of items bought for each customer's transaction. Reminder that each Index (row) represents a customer's transaction

In [28]:
grocery2.sum(axis=1)

0     4
1     3
2     4
3     3
4     3
5     3
6     4
7     4
8     2
9     4
10    4
11    3
12    3
13    3
14    3
15    3
16    3
17    3
18    4
dtype: int64

##### After I have completed my data cleaning, I will now use the Apriori rule to calculate my support for each item. My support essentially represents the frequency (in the form of a percentage) that each item is purchased and ranges from values 0 to 1. 

In [35]:
groc_support=apriori(grocery2, min_support=.1, use_colnames=True)
groc_support.sort_values(by = 'support', ascending = False)

,support,itemsets
2,0.631579,(BREAD)
4,0.421053,(COFFEE)
10,0.368421,(TEA)
0,0.315789,(BISCUIT)
5,0.315789,(CORNFLAKES)
9,0.315789,(SUGER)
7,0.263158,(MAGGI)
24,0.210526,"(BREAD, SUGER)"
25,0.210526,"(BREAD, TEA)"
28,0.210526,"(COFFEE, CORNFLAKES)"


##### Just a few key comments:
- Bread is the most frequent bought item at 63% of baskets
- Jam is the least most bought single item at 11% of baskets
- Bread and Sugar were the most frequent bought combination at 21%
- Coffee was the most frequent bought beverage at 42%
    
##### Depending on business need, there are many more combinations and analysis that you can pull from this dataset, just by exploring through support, confidence, and lift.

##### Once I obtain my support for each item and itemset, I can now apply the association rule to determine the support, confidence, and lift. I have set the nmin_threshold for my lift column at 1.5 because I want to only show items and itemsets that we strongly feel are associated and not just together by random chance. 

##### These metrics and min_threshold values can be changed at any time to explore the data even more 

In [51]:
groc_association=association_rules(groc_support, metric='lift', min_threshold=1.5)
groc_association.sort_values(by='lift', ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
75,"(COFFEE, BISCUIT)","(CORNFLAKES, COCK)",0.105263,0.105263,0.105263,1.000000,9.500000,0.094183,inf
78,"(CORNFLAKES, COCK)","(COFFEE, BISCUIT)",0.105263,0.105263,0.105263,1.000000,9.500000,0.094183,inf
58,(JAM),"(BREAD, MAGGI)",0.105263,0.157895,0.105263,1.000000,6.333333,0.088643,inf
22,"(COFFEE, BISCUIT)",(COCK),0.105263,0.157895,0.105263,1.000000,6.333333,0.088643,inf
70,"(COFFEE, CORNFLAKES, BISCUIT)",(COCK),0.105263,0.157895,0.105263,1.000000,6.333333,0.088643,inf
...,...,...,...,...,...,...,...,...,...
6,(BREAD),(JAM),0.631579,0.105263,0.105263,0.166667,1.583333,0.038781,1.073684
5,(SUGER),(BOURNVITA),0.315789,0.210526,0.105263,0.333333,1.583333,0.038781,1.184211
39,(BISCUIT),"(COFFEE, CORNFLAKES)",0.315789,0.210526,0.105263,0.333333,1.583333,0.038781,1.184211
36,"(CORNFLAKES, BISCUIT)",(COFFEE),0.157895,0.421053,0.105263,0.666667,1.583333,0.038781,1.736842


##### Just a few key comments:
- Given that a customer has 'Jam' in their basket, we are strongly confident that they will add "Bread' and 'Maggi'
- Given that someone has 'Bread' in there basket, we are not as confident that they will add 'Jam'
- Given that a customer has 'Sugar' in their basket, we are fairly confident that they will add 'Coffee'
- Bread is the most frequent bought item
    
##### Depending on business need, there are many more combinations and analysis that you can pull from this dataset, just by exploring through support, confidence, and lift.
    

Thank you for reviewing my notebook and please feel free to reach me through any of the contact formats below:

- GitHub: github.com/CarlosBusiness
- LinkedIN: www.linkedin.com/in/CarlosJonesData
- Twitter: Los The Scientist
- Email: carlosjones.business@gmail.com